# Enhancing our pipeline to have better prediction for "Target" 

As saw in previous steps, the best models for our binary prediction of "Target" are:

The next step is to refine the parameters choice. We will apply grid search on each selected model to obtain the best configuration for each.

After that, we will combine the models in a hybrid ensemble using a VotingClassifier